### Note:
we tried to implement CUDA processing to fasten inference time and permit larger datasets, but with no success. \
As SBI documentation says SNPE does not benefit from CUDA operations.

In [1]:
import time
import os
import torch
from sbi import utils as utils
from sbi.inference import SNPE, SNPE_A, SNPE_C
from sbi.utils.user_input_checks import process_prior

from parsing_functions import save_pickle_data, load_inference_data, extract_specs, preprocess

## Loading dataset

In [2]:
save_dir = 'saved_datasets/'
dataset_name = 'dataset_eb1fe5f16d9ec420409495714fb8d7a8.pickle'

dataset_path = os.path.join(save_dir, dataset_name)

xstack, theta = load_inference_data(input_file = dataset_path)

#print(xstack.shape)

# Here you can choose the features used for inference from: 'Cxx','S_red_x','Cxy','Cyy','S_red_y'
features = ['Cxx','S_red_x','Cxy','Cyy','S_red_y']
x = preprocess(data = xstack, features = features ,density = 1.)

#print(type(x), type(xstack))

# Extract the specifications
specs_dict = extract_specs(dataset_path)

# Unpack the specifications from the dictionary
num_simulations = specs_dict['num_simulations']
Npts = specs_dict['Npts']
dt = specs_dict['dt']
oversampling = specs_dict['oversampling']
prerun = specs_dict['prerun']

high_tensor = specs_dict['high_tensor']
low_tensor = specs_dict['low_tensor']

# Declaring SBI prior for sampling
device = torch.device("cpu")
prior_sbi = utils.BoxUniform(low=low_tensor.to(device), 
                             high=high_tensor.to(device),
                            device = 'cpu')

# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior_sbi)

## Inference

In [3]:
#print(torch.cuda.is_available())
#print(torch.cuda.get_device_name(0))

#torch.cuda.empty_cache()
#print(torch.cuda.memory_summary())

In [4]:
inference = SNPE(prior=prior, device="cpu", density_estimator="maf")
inference = inference.append_simulations(theta, x)
start = time.time()
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)
end = time.time()
print('\n time passed for convergence: ', end-start)

 Neural network successfully converged after 124 epochs.
 time passed for convergence:  23.799218893051147


In [5]:
#saving generated posterior from SBI
posterior_dir = 'saved_posteriors/'

data_posterior = {
    'posterior': posterior,
    'num_simulations': num_simulations,
    'Npts': Npts,
    'dt': dt,
    'oversampling': oversampling,
    'prerun': prerun,
    'low_tensor': low_tensor,
    'high_tensor': high_tensor,
    'features': features # Indicates the type of the data
}

save_pickle_data(data=data_posterior, folder_path=posterior_dir)

Saved dataset at saved_posteriors/posterior_366eb24ca0e06eb1e96ac05dfd2df8da.pickle



'saved_posteriors/posterior_366eb24ca0e06eb1e96ac05dfd2df8da.pickle'